In [63]:
from sqlalchemy import create_engine
import pandas as pd
from time import time
import argparse
import os
import requests
import wget
from io import StringIO

In [64]:
user = 'root'
password = 'root'
host = 'localhost'
port = 5432
db = 'ny_taxi'
table_name = 'yellow_taxi_trips'
url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv"

csv_name=requests.get(url).text
# df = pd.read_csv(StringIO(csv_name))
# df.head()

# from io import StringIO

# import pandas as pd
# import requests
# url='https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv'
# s=requests.get(url).text

# c=pd.read_csv(StringIO(s))

In [65]:
# url="https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv"

df_iter = pd.read_csv(StringIO(csv_name),iterator=True ,chunksize=100000)
engine  = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

# df_iter = pd.read_csv(csv_name,iterator=True ,chunksize=100000)
df = next(df_iter)

df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append') 
    
while True:
    t_start = time()
    df = next(df_iter)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append')
    
    t_end = time()
    
    print('inserted another chunk..., took %.3f second' %(t_end - t_start))

inserted another chunk..., took 21.083 second
inserted another chunk..., took 20.888 second
inserted another chunk..., took 23.439 second
inserted another chunk..., took 43.365 second
inserted another chunk..., took 32.864 second
inserted another chunk..., took 23.683 second
inserted another chunk..., took 25.398 second
inserted another chunk..., took 383.905 second
inserted another chunk..., took 35.074 second
inserted another chunk..., took 21.951 second
inserted another chunk..., took 22.156 second


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


inserted another chunk..., took 21.957 second
inserted another chunk..., took 14.696 second


StopIteration: 